In [176]:
import pandas as pd
import requests
from faker import Faker
import re
import phonenumbers
import numpy as np

import sqlite3

In [1]:
!pip install phonenumbers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 17.0 MB/s eta 0:00:00m eta 0:00:010:00:01


In [87]:
#Extract data from API

response= requests.get('https://jsonplaceholder.typicode.com/users')
users = response.json()

In [88]:
users_df = pd.DataFrame(users)
users_df_copy = users_df.copy()
users_df

,id,name,username,email,address,phone,website,company
0,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu..."
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac..."
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ..."
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult..."
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c..."
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '..."
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config..."
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im..."
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit..."
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central..."


In [74]:
#normalize address column
df_address = pd.json_normalize(users_df['address'])

#union of the two dataframes
users_df_test = pd.concat([users_df, df_address], axis=1)

users_df_test
df_resultado = users_df_test.drop('address', axis=1)

df_resultado

,id,name,username,email,phone,website,company,street,suite,city,zipcode,geo.lat,geo.lng
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu...",Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.3159,81.1496
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac...",Victor Plains,Suite 879,Wisokyburgh,90566-7771,-43.9509,-34.4618
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,1-463-123-4447,ramiro.info,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ...",Douglas Extension,Suite 847,McKenziehaven,59590-4157,-68.6102,-47.0653
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult...",Hoeger Mall,Apt. 692,South Elvis,53919-4257,29.4572,-164.2990
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c...",Skiles Walks,Suite 351,Roscoeview,33263,-31.8129,62.5342
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '...",Norberto Crossing,Apt. 950,South Christy,23505-1337,-71.4197,71.7478
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config...",Rex Trail,Suite 280,Howemouth,58804-1099,24.8918,21.8984
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im...",Ellsworth Summit,Suite 729,Aliyaview,45169,-14.3990,-120.7677
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit...",Dayna Park,Suite 449,Bartholomebury,76495-3109,24.6463,-168.8889
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,024-648-3804,ambrose.net,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central...",Kattie Turnpike,Suite 198,Lebsackbury,31428-2261,-38.2386,57.2232


In [5]:
users_test = pd.read_csv('users_extended.csv')
users_test


,USER_ID,USER_NAME,USER_USERNAME,USER_EMAIL,USER_PHONE,WEBSITE,USER_STREET,USER_SUITE,USER_CITY,USER_ZIPCODE,USER_LAT,USER_LNG
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org,Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.315900,81.149600
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net,Victor Plains,Suite 879,Wisokyburgh,90566-7771,-43.950900,-34.461800
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,1-463-123-4447,ramiro.info,Douglas Extension,Suite 847,McKenziehaven,59590-4157,-68.610200,-47.065300
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,493-170-9623 x156,kale.biz,Hoeger Mall,Apt. 692,South Elvis,53919-4257,29.457200,-164.299000
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,(254)954-1289,demarco.info,Skiles Walks,Suite 351,Roscoeview,33263,-31.812900,62.534200
...,...,...,...,...,...,...,...,...,...,...,...,...
55005,55006,Alexander Hughes,hamiltonthomas,martinezvictoria@example.net,846.209.1759x957,https://www.campbell.com/,525 Jackson Port,Apt. 878,North Stephanieton,19518,39.375597,97.178865
55006,55007,Paul Kim,robertrussell,staffordwilliam@example.net,325.916.7235x093,http://www.townsend.info/,526 Dennis Harbor Apt. 889,Apt. 691,New Marystad,01913,-22.499807,34.631837
55007,55008,Elizabeth Brewer,mercadolisa,charles49@example.org,001-804-520-9188x5650,http://brown.com/,149 Figueroa Walks Apt. 711,Suite 269,Lake Amandaland,57758,-25.364915,-140.349789
55008,55009,Amanda Underwood,wsimmons,jennifer14@example.org,625-816-9453,https://dean-jones.com/,8959 Karen Glen,Suite 848,West Brent,45819,-5.524719,-176.450230


In [25]:


# Program to get timezone a phone number 
  
import phonenumbers 
from phonenumbers import timezone 
  
# Parsing String to Phone number 
phoneNumber = phonenumbers.parse("+919876543210") 
print(len(str(phoneNumber.national_number)))
  
# Pass the parsed phone number in below function 
timeZone = timezone.time_zones_for_number(phoneNumber) 
  
# It print the timezone of a phonenumber 
print(timeZone)


10
('Asia/Calcutta',)


In [26]:
def format_phone_number(phone_number):
    try:
        # Parse el número de teléfono utilizando la biblioteca phonenumbers
        parsed_number = phonenumbers.parse(phone_number, None)

        # Obtenga el formato nacional del número de teléfono
        national_format = phonenumbers.format_number(parsed_number, phonenumbers.PhoneNumberFormat.NATIONAL)

        # Compruebe las condiciones según las instrucciones proporcionadas
        if len(str(parsed_number.national_number)) == 9:
            # Números de teléfono de 9 dígitos: guardar tal cual
            return phone_number
        elif len(str(parsed_number.national_number)) == 10:
            if str(parsed_number.national_number)[0] == '1':
                # Números de teléfono de 10 dígitos que comienzan con 1: formato con paréntesis
                return national_format
            else:
                # Números de teléfono de 10 dígitos que no comienzan con 1: formato con paréntesis
                return national_format
        elif len(str(parsed_number.national_number)) == 11:
            if str(parsed_number.national_number)[0] == '1':
                # Números de teléfono de 11 dígitos que comienzan con 1: quitar el primer dígito y formato con paréntesis
                return national_format[1:]
            else:
                # Números de teléfono de 11 dígitos que no comienzan con 1: guardar tal cual
                return phone_number
        else:
            # Otros casos: guardar tal cual
            return phone_number
    except phonenumbers.NumberParseException:
        # Si no se puede analizar el número de teléfono, guárdelo tal cual
        return phone_number

In [27]:
users_test['USER_PHONE_FORMATTED'] = users_test['USER_PHONE'].apply(format_phone_number)

# Imprimir el DataFrame resultante
print(users_test)

       USER_ID         USER_NAME   USER_USERNAME  \
0            1     Leanne Graham            Bret   
1            2      Ervin Howell       Antonette   
2            3  Clementine Bauch        Samantha   
3            4  Patricia Lebsack        Karianne   
4            5  Chelsey Dietrich          Kamren   
...        ...               ...             ...   
55005    55006  Alexander Hughes  hamiltonthomas   
55006    55007          Paul Kim   robertrussell   
55007    55008  Elizabeth Brewer     mercadolisa   
55008    55009  Amanda Underwood        wsimmons   
55009    55010      Erica Newman         angel66   

                         USER_EMAIL             USER_PHONE  \
0                 Sincere@april.biz  1-770-736-8031 x56442   
1                 Shanna@melissa.tv    010-692-6593 x09125   
2                Nathan@yesenia.net         1-463-123-4447   
3         Julianne.OConner@kory.org      493-170-9623 x156   
4          Lucio_Hettinger@annie.ca          (254)954-1289   
...

In [28]:
users_test

,USER_ID,USER_NAME,USER_USERNAME,USER_EMAIL,USER_PHONE,WEBSITE,USER_STREET,USER_SUITE,USER_CITY,USER_ZIPCODE,USER_LAT,USER_LNG,USER_PHONE_FORMATTED
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org,Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.315900,81.149600,1-770-736-8031 x56442
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net,Victor Plains,Suite 879,Wisokyburgh,90566-7771,-43.950900,-34.461800,010-692-6593 x09125
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,1-463-123-4447,ramiro.info,Douglas Extension,Suite 847,McKenziehaven,59590-4157,-68.610200,-47.065300,1-463-123-4447
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,493-170-9623 x156,kale.biz,Hoeger Mall,Apt. 692,South Elvis,53919-4257,29.457200,-164.299000,493-170-9623 x156
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,(254)954-1289,demarco.info,Skiles Walks,Suite 351,Roscoeview,33263,-31.812900,62.534200,(254)954-1289
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55005,55006,Alexander Hughes,hamiltonthomas,martinezvictoria@example.net,846.209.1759x957,https://www.campbell.com/,525 Jackson Port,Apt. 878,North Stephanieton,19518,39.375597,97.178865,846.209.1759x957
55006,55007,Paul Kim,robertrussell,staffordwilliam@example.net,325.916.7235x093,http://www.townsend.info/,526 Dennis Harbor Apt. 889,Apt. 691,New Marystad,01913,-22.499807,34.631837,325.916.7235x093
55007,55008,Elizabeth Brewer,mercadolisa,charles49@example.org,001-804-520-9188x5650,http://brown.com/,149 Figueroa Walks Apt. 711,Suite 269,Lake Amandaland,57758,-25.364915,-140.349789,001-804-520-9188x5650
55008,55009,Amanda Underwood,wsimmons,jennifer14@example.org,625-816-9453,https://dean-jones.com/,8959 Karen Glen,Suite 848,West Brent,45819,-5.524719,-176.450230,625-816-9453


In [36]:
import pandas as pd
from faker import Faker

# Supongamos que ya tienes un DataFrame llamado df_companies con las columnas mencionadas

# Crear un ejemplo de DataFrame
data_companies = {
    'USER_ID': range(1, 11),
    'COMPANY_NAME': ['Romaguera-Crona', 'Deckow-Crist', 'Romaguera-Jacobson', 'Robel-Corkery', 'Keebler LLC', 'Considine-Lockman', 'Johns Group', 'Abernathy Group', 'Yost and Sons', 'Hoeger LLC'],
    'COMPANY_CATCHPHRASE': ['Multi-layered client-server neural-net', 'Proactive didactic contingency', 'Face to face bifurcated interface', 'Multi-tiered zero tolerance productivity', 'User-centric fault-tolerant solution', 'Synchronised bottom-line interface', 'Configurable multimedia task-force', 'Implemented secondary concept', 'Switchable contextually-based project', 'Centralized empowering task-force'],
    'COMPANY_BS': ['harness real-time e-markets', 'synergize scalable supply-chains', 'e-enable strategic applications', 'transition cutting-edge web services', 'revolutionize end-to-end systems', 'e-enable innovative applications', 'generate enterprise e-tailers', 'e-enable extensible e-tailers', 'aggregate real-time technologies', 'target end-to-end models']
}

df_companies = pd.DataFrame(data_companies)

# Crear un objeto Faker
fake = Faker()

# Obtener 30,000 user_id únicos entre 1 y 30000
unique_user_ids = set()
while len(unique_user_ids) < 45:
    unique_user_ids.add(fake.random_int(min=11, max=60))

# Crear 30,000 registros adicionales con user_id únicos
for user_id in unique_user_ids:
    # Generar datos aleatorios usando Faker
    company_name = fake.company()
    company_catchphrase = fake.catch_phrase()
    company_bs = fake.bs()

    # Crear un nuevo registro
    new_record = {
        'USER_ID': user_id,
        'COMPANY_NAME': company_name,
        'COMPANY_CATCHPHRASE': company_catchphrase,
        'COMPANY_BS': company_bs
    }
    
    new_company_df = pd.DataFrame([new_record])

    df_companies = pd.concat([df_companies, new_company_df], ignore_index=True)


# Imprimir el DataFrame resultante
print(df_companies)


    USER_ID                    COMPANY_NAME  \
0         1                 Romaguera-Crona   
1         2                    Deckow-Crist   
2         3              Romaguera-Jacobson   
3         4                   Robel-Corkery   
4         5                     Keebler LLC   
5         6               Considine-Lockman   
6         7                     Johns Group   
7         8                 Abernathy Group   
8         9                   Yost and Sons   
9        10                      Hoeger LLC   
10       12                    Blake-Wright   
11       13                     Coleman Ltd   
12       14  Delacruz, Sweeney and Mcdonald   
13       15   Navarro, Clayton and Martinez   
14       16                      Fowler Ltd   
15       17                    Garrison Inc   
16       18                       Parks LLC   
17       19                Martinez-Freeman   
18       20       Anderson, Combs and Moran   
19       21                      Peters LLC   
20       22  

In [37]:
unique_user_ids = set()
while len(unique_user_ids) < 30:
    unique_user_ids.add(fake.random_int(min=1, max=50))
    
    
unique_user_ids

{1,
 3,
 4,
 5,
 6,
 8,
 10,
 11,
 15,
 16,
 17,
 18,
 20,
 22,
 24,
 25,
 27,
 29,
 30,
 31,
 35,
 36,
 38,
 39,
 41,
 44,
 45,
 46,
 47,
 48}

In [38]:
unique_company_ids = range(1, 30 + 1)

unique_company_ids

range(1, 31)

In [55]:
data_companies = {
    'COMPANY_ID': range(1, 11),
    'USER_ID': range(1, 11),
    'COMPANY_NAME': ['Romaguera-Crona', 'Deckow-Crist', 'Romaguera-Jacobson', 'Robel-Corkery', 'Keebler LLC', 'Considine-Lockman', 'Johns Group', 'Abernathy Group', 'Yost and Sons', 'Hoeger LLC'],
    'COMPANY_CATCHPHRASE': ['Multi-layered client-server neural-net', 'Proactive didactic contingency', 'Face to face bifurcated interface', 'Multi-tiered zero tolerance productivity', 'User-centric fault-tolerant solution', 'Synchronised bottom-line interface', 'Configurable multimedia task-force', 'Implemented secondary concept', 'Switchable contextually-based project', 'Centralized empowering task-force'],
    'COMPANY_BS': ['harness real-time e-markets', 'synergize scalable supply-chains', 'e-enable strategic applications', 'transition cutting-edge web services', 'revolutionize end-to-end systems', 'e-enable innovative applications', 'generate enterprise e-tailers', 'e-enable extensible e-tailers', 'aggregate real-time technologies', 'target end-to-end models']
}

df_companies = pd.DataFrame(data_companies)

In [74]:
import pandas as pd
from faker import Faker

# Supongamos que ya tienes un DataFrame llamado df_companies con las columnas mencionadas

# Crear un objeto Faker
fake = Faker()

# Obtener los USER_ID y COMPANY_ID únicos actuales
existing_user_ids = set(df_companies['USER_ID'].unique())
existing_company_ids = set(df_companies['COMPANY_ID'].unique())

# Crear una lista para almacenar los nuevos registros
new_records = []

# Generar 30,000 registros adicionales
for _ in range(30000):
    # Generar USER_ID único
    user_id = fake.random_int(min=1, max=55010)
    while user_id in existing_user_ids:
        user_id = fake.random_int(min=1, max=55010)

    # Generar COMPANY_ID único
    company_id = max(existing_company_ids) + 1
    while company_id in existing_company_ids:
        company_id += 1

    # Actualizar los conjuntos de USER_ID y COMPANY_ID existentes
    existing_user_ids.add(user_id)
    existing_company_ids.add(company_id)

    # Generar datos aleatorios usando Faker
    company_name = fake.company()
    company_catchphrase = fake.catch_phrase()
    company_bs = fake.bs()

    # Crear un nuevo registro
    new_record = {
        'COMPANY_ID': company_id,
        'USER_ID': user_id,
        'COMPANY_NAME': company_name,
        'COMPANY_CATCHPHRASE': company_catchphrase,
        'COMPANY_BS': company_bs
    }

    # Agregar el nuevo registro a la lista
    new_records.append(new_record)

# Concatenar los nuevos registros al DataFrame existente
df_companies_extended = pd.concat([df_companies, pd.DataFrame(new_records)], ignore_index=True)

# Imprimir el DataFrame resultante
print(df_companies_extended)



       COMPANY_ID  USER_ID                   COMPANY_NAME  \
0               1        1                Romaguera-Crona   
1               2        2                   Deckow-Crist   
2               3        3             Romaguera-Jacobson   
3               4        4                  Robel-Corkery   
4               5        5                    Keebler LLC   
...           ...      ...                            ...   
30005       30006      303                    Perkins PLC   
30006       30007    34132                   Marshall PLC   
30007       30008     6935                 Griffin-Wilson   
30008       30009    32384  Reynolds, Bautista and Molina   
30009       30010    49626        Lynch, Adkins and Baker   

                             COMPANY_CATCHPHRASE  \
0         Multi-layered client-server neural-net   
1                 Proactive didactic contingency   
2              Face to face bifurcated interface   
3       Multi-tiered zero tolerance productivity   
4      

In [80]:
import pandas as pd
from faker import Faker

def random_companies(quantity_users, quantity_companies, df_companies):
    #faker object
    fake = Faker()

    #obtain current unique USER_ID and COMPANY_ID
    existing_user_ids = set(df_companies['USER_ID'].unique())
    existing_company_ids = set(df_companies['COMPANY_ID'].unique())

    new_records = []

    #generate additional records
    for _ in range(quantity_companies):
        #generate unique USER_ID
        user_id = fake.random_int(min=1, max=quantity_users)
        while user_id in existing_user_ids:
            user_id = fake.random_int(min=1, max=quantity_users)

        #generate unique COMPANY_ID
        company_id = max(existing_company_ids) + 1
        while company_id in existing_company_ids:
            company_id += 1

        #update existing USER_ID and COMPANY_ID sets
        existing_user_ids.add(user_id)
        existing_company_ids.add(company_id)

        #generate random data
        company_name = fake.company()
        company_catchphrase = fake.catch_phrase()
        company_bs = fake.bs()

        #create new record
        new_record = {
            'COMPANY_ID': company_id,
            'USER_ID': user_id,
            'COMPANY_NAME': company_name,
            'COMPANY_CATCHPHRASE': company_catchphrase,
            'COMPANY_BS': company_bs
        }

        new_records.append(new_record)

    #concatenate new records to existing DataFrame
    df_companies_extended = pd.concat([df_companies, pd.DataFrame(new_records)], ignore_index=True)

    #sort DataFrame by 'COMPANY_ID'
    df_companies_extended.sort_values(by='COMPANY_ID', inplace=True)

    #save DataFrame to a CSV file
    df_companies_extended.to_csv('companies_extended.csv', index=False)

    return df_companies_extended

In [106]:
booking_df= pd.read_csv('raw_bookings.csv', sep=';')
booking_df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date
0,Resort Hotel,0,109,2016,January,1,0,1,2,0,BB,RUS,0,0,0,A,D,2.0,Check-Out,02/01/2016
1,Resort Hotel,0,109,2016,January,1,0,1,2,2,BB,RUS,0,0,0,H,H,2.0,Check-Out,02/01/2016
2,Resort Hotel,1,2,2016,January,1,0,1,2,0,BB,PRT,0,0,0,D,D,2.0,No-Show,01/01/2016
3,Resort Hotel,0,88,2016,January,1,0,2,2,0,HB,ARG,0,0,0,A,D,2.0,Check-Out,03/01/2016
4,Resort Hotel,1,20,2016,January,1,0,2,2,2,BB,PRT,0,0,0,G,G,2.0,Canceled,22/12/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,City Hotel,0,17,2016,December,30,2,5,2,0,SC,FRA,0,0,0,A,A,9.0,Check-Out,06/01/2017
56703,City Hotel,0,177,2016,December,30,2,5,0,0,BB,PRT,0,0,0,E,K,NaN,Check-Out,06/01/2017
56704,City Hotel,0,107,2016,December,31,2,5,2,0,BB,FRA,0,0,0,A,B,2.0,Check-Out,07/01/2017
56705,City Hotel,0,310,2016,December,27,2,10,2,0,SC,DEU,0,0,0,A,A,9.0,Check-Out,08/01/2017


In [108]:
def bookings_extended(quantity_users, booking_df):
    # Crear un objeto Faker
    fake = Faker()
    
    df = pd.DataFrame()
    # Agregar columna 'USER_ID' con valores aleatorios entre 1 y 55010
    df['USER_ID'] = [fake.random_int(min=1, max=quantity_users) for i in range(len(df_reservas))]

    # Agregar columna 'LOCALIZADOR_RESERVA' con valores únicos
    booking_codes = set()
    
    while len(booking_codes) < len(booking_df):
        booking_code = fake.vin()
        booking_codes.add(booking_code)

    df['BOOKING_CODE'] = list(booking_codes)
    
    booking_df_extended = pd.concat([df,booking_df], axis=1)


    return booking_df_extended


In [109]:
test = bookings_extended(55010, booking_df)
test

,USER_ID,BOOKING_CODE,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,meal,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date
0,13889,PR1ZYAFL3H04H84W9,Resort Hotel,0,109,2016,January,1,0,1,...,BB,RUS,0,0,0,A,D,2.0,Check-Out,02/01/2016
1,38752,EYUF5YLM8STBHW5KS,Resort Hotel,0,109,2016,January,1,0,1,...,BB,RUS,0,0,0,H,H,2.0,Check-Out,02/01/2016
2,37102,ZERSC2F24US6K9ZEE,Resort Hotel,1,2,2016,January,1,0,1,...,BB,PRT,0,0,0,D,D,2.0,No-Show,01/01/2016
3,7613,NFXW1P5Z9N0YVYGDX,Resort Hotel,0,88,2016,January,1,0,2,...,HB,ARG,0,0,0,A,D,2.0,Check-Out,03/01/2016
4,636,HAZTV34W08616MNB8,Resort Hotel,1,20,2016,January,1,0,2,...,BB,PRT,0,0,0,G,G,2.0,Canceled,22/12/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,34676,F36770NS27LF4DB1H,City Hotel,0,17,2016,December,30,2,5,...,SC,FRA,0,0,0,A,A,9.0,Check-Out,06/01/2017
56703,16107,C1T6NRHR9XSF8J6RG,City Hotel,0,177,2016,December,30,2,5,...,BB,PRT,0,0,0,E,K,NaN,Check-Out,06/01/2017
56704,19557,UBUGXXHL43BK0X3LE,City Hotel,0,107,2016,December,31,2,5,...,BB,FRA,0,0,0,A,B,2.0,Check-Out,07/01/2017
56705,18152,VNSDZKN9X1GW5T5B6,City Hotel,0,310,2016,December,27,2,10,...,SC,DEU,0,0,0,A,A,9.0,Check-Out,08/01/2017


In [220]:
users_df= pd.read_csv('raw_users.csv')
users_df

,id,name,username,email,address,phone,website,company
0,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu..."
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac..."
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ..."
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult..."
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c..."
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '..."
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config..."
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im..."
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit..."
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central..."


In [221]:
def create_company_id(users_df):
    users_df['company_id'] = np.random.permutation(np.arange(1, len(users_df) + 1))
    
def random_users(quantity, users_df):
    '''
        Function to extend the number of users. 
        It can only be used with the users table because it is specific to it 
        and it is only used once in order to increase the number of users that 
        have made reservations in the hotels and to have consistency in the data.
        Also, it create companies id

        Inputs:

        df_users = users DataFrame
        quantity = number of users to be added 
    '''
    fake = Faker()

    #number of new users
    additional_users = quantity

    new_users = []

    #create random users
    for i in range(additional_users):

        if i == 0:
            user_id = users_df['id'].max() + 1
        else:
            user_id = new_users[i - 1]['id'] + 1

        name = fake.name()
        username = fake.user_name()
        email = fake.email()
        street = fake.street_name()
        suite = fake.random_element(elements=('Apt. 101', 'Suite 202', 'Room 303'))
        city = fake.city()
        zipcode = fake.zipcode()
        lat = fake.latitude()
        lng = fake.longitude()
        phone = fake.phone_number()
        website = fake.url()
        company_name = fake.company()
        company_catchphrase = fake.catch_phrase()
        bs = fake.bs()

        new_record = {
            'id': user_id,
            'name': name,
            'username': username,
            'email': email,
            'address': {
                'street': street,
                'suite': suite,
                'city': city,
                'zipcode': zipcode,
                'geo': {'lat': lat, 'lng': lng}
            },
            'phone': phone,
            'website': website,
            'company': {
                'name': company_name,
                'catchPhrase': company_catchphrase,
                'bs': bs
            }
        }

        #add new record to the list
        new_users.append(new_record)

    #concatenate two dataframes
    users_df_extended = pd.concat([users_df, pd.DataFrame(new_users)], ignore_index=True)
    
    create_company_id(users_df_extended)

    return users_df_extended

In [222]:
users_test = random_users(40, users_df)

In [223]:
users_test

,id,name,username,email,address,phone,website,company,company_id
0,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu...",6
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac...",2
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ...",25
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult...",29
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c...",44
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '...",30
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config...",41
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im...",39
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit...",42
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central...",13


In [224]:
def create_id(df, column_id, quantity=None):
    '''
    creation of random companies id in dataframe
    '''
    if quantity:
        df[column_id] = [fake.random_int(min=1, max=quantity_users) for i in range(len(df))]
    else:
        df[column_id] = np.random.permutation(np.arange(1, len(df) + 1))
        
        
        
def create_booking_id(booking_df):
    
    '''
    Function to add to the df of reservations, the codes of bookings of 
    each record.
    
    inputs:
    booking_df = df of reserves
    '''
    #faker object
    fake = Faker()
    
    #empty df
    df = pd.DataFrame()
    
    #empty booking set
    booking_codes = set()
    
    while len(booking_codes) < len(booking_df):
        booking_code = fake.vin()
        booking_codes.add(booking_code)
    
    #create booking_code column
    df['BOOKING_CODE'] = list(booking_codes)
    
    #union of two df
    booking_with_codes_df = pd.concat([df,booking_df], axis=1)

    return booking_with_codes_df

In [225]:
users_test = create_booking_id(users_test)

In [226]:
create_id(users_test, 'test2_id', 20)

In [227]:
users_test

,BOOKING_CODE,id,name,username,email,address,phone,website,company,company_id,test2_id
0,41DUBNM48VSCNEGTN,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu...",6,17
1,0KYFCF6T2XU1M8Z5C,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac...",2,14
2,JHYSAP889W8EAPXDY,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ...",25,18
3,RABKL9SU6VGST7BAT,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult...",29,8
4,WCRR8BTT4MV6B414X,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c...",44,7
5,UZLZ7H0S3JDYY6G2K,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '...",30,6
6,ENMEHSEF9UHKMH647,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config...",41,5
7,LXK7S18X4FYDM9ZNY,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im...",39,5
8,7MKF9EX554VSLSS5U,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit...",42,8
9,BFYUDFNGXWUXM2567,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central...",13,12


In [228]:
def only_some_records(percentage, df, column_id, column_to_modify):
    
    #number of rows to modify
    num_rows_to_modify = int(percentage * len(df))
    print("estas son la cantidad de filas a modificar")
    print(num_rows_to_modify)
    #obtain the highest ids to be modified
    id_to_modify = df.nlargest(num_rows_to_modify, column_id)[column_id]
    print("estos son los id a modificar")
    print(type(id_to_modify))
    print(id_to_modify)
    df.loc[df[column_id].isin(id_to_modify), [column_id, column_to_modify]] = [np.nan, np.nan]
    print(df[column_id].isin(id_to_modify))
    print([column_id, column_to_modify])
    

In [230]:
only_some_records(0.4, users_test, 'test2_id', 'company')

estas son la cantidad de filas a modificar
20
estos son los id a modificar
<class 'pandas.core.series.Series'>
21    19
24    19
25    19
39    19
2     18
10    18
0     17
12    17
13    17
34    16
18    15
33    15
1     14
14    14
16    14
9     12
29    11
23    10
28     9
30     9
Name: test2_id, dtype: int64
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
Name: test2_id, dtype: bool
['test2_id', 'company']


In [207]:
users_test

,BOOKING_CODE,id,name,username,email,address,phone,website,company,company_id,test_id,test2_id
0,2V05UM5B0LJGP97UR,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,NaN,14,NaN,9
1,00J4N2MU7GN7ARA0U,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac...",49,19.0,14
2,T03AZACE1D0K8R5WU,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ...",25,3.0,12
3,Z7J78NBL50XCBSYGL,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult...",12,30.0,15
4,DE40KK5N6E9MCUY76,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,NaN,21,NaN,17
5,RPRN43M80FCTMLJAK,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,NaN,10,NaN,6
6,JSCDNWGY7BLXWV7HZ,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config...",45,21.0,9
7,4RC9UL991F0VKY0D4,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im...",48,11.0,5
8,GPBY17NH9G79M7MLE,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit...",42,14.0,13
9,JJ0GHB8R0N9TG2Y3K,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central...",37,26.0,9


In [233]:
import pandas as pd
import pycountry

# Crear un DataFrame de ejemplo
data = {'codigo_pais': ['RUS', 'PRT', 'ARG', 'USA', 'FRA', 'ESP']}
df = pd.DataFrame(data)

# Función para obtener el nombre del país a partir del código de país
def get_country_name(code):
    try:
        country = pycountry.countries.get(alpha_3=code)
        return country.name
    except AttributeError:
        return None

# Aplicar la función para crear una nueva columna 'pais'
df['pais'] = df['codigo_pais'].apply(get_country_name)

# Imprimir el DataFrame resultante
print(df)


  codigo_pais                pais
0         RUS  Russian Federation
1         PRT            Portugal
2         ARG           Argentina
3         USA       United States
4         FRA              France
5         ESP               Spain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 33.9 MB/s eta 0:00:00m eta 0:00:01:01:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681832 sha256=fef44ec0de9b735ae412b1b412ac6f3f8694ddb8c62f361975b9745b0e64164f
  Stored in directory: /Users/carlasanchezvalles/Library/Caches/pip/wheels/62/4a/9c/7a46699df9efb845aa116fae5e52d8690fc442fef6d32213f7
Successfully built pycountry


In [239]:
bookings= pd.read_csv('raw_bookings.csv', sep=';')
bookings

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date
0,Resort Hotel,0,109,2016,January,1,0,1,2,0,BB,RUS,0,0,0,A,D,2.0,Check-Out,02/01/2016
1,Resort Hotel,0,109,2016,January,1,0,1,2,2,BB,RUS,0,0,0,H,H,2.0,Check-Out,02/01/2016
2,Resort Hotel,1,2,2016,January,1,0,1,2,0,BB,PRT,0,0,0,D,D,2.0,No-Show,01/01/2016
3,Resort Hotel,0,88,2016,January,1,0,2,2,0,HB,ARG,0,0,0,A,D,2.0,Check-Out,03/01/2016
4,Resort Hotel,1,20,2016,January,1,0,2,2,2,BB,PRT,0,0,0,G,G,2.0,Canceled,22/12/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,City Hotel,0,17,2016,December,30,2,5,2,0,SC,FRA,0,0,0,A,A,9.0,Check-Out,06/01/2017
56703,City Hotel,0,177,2016,December,30,2,5,0,0,BB,PRT,0,0,0,E,K,NaN,Check-Out,06/01/2017
56704,City Hotel,0,107,2016,December,31,2,5,2,0,BB,FRA,0,0,0,A,B,2.0,Check-Out,07/01/2017
56705,City Hotel,0,310,2016,December,27,2,10,2,0,SC,DEU,0,0,0,A,A,9.0,Check-Out,08/01/2017


In [275]:
import pandas as pd
import pycountry


# Función para obtener el nombre del país a partir del código de país
import pycountry

def get_country_name(code):
    try:
        country = pycountry.countries.get(alpha_3=code)
        if country:
            return country.name
        else:
            country = pycountry.countries.get(alpha_2=code)
            return country.name
    except (LookupError, AttributeError):
        return np.nan



In [276]:
bookings['country'].apply(get_country_name)

0        Russian Federation
1        Russian Federation
2                  Portugal
3                 Argentina
4                  Portugal
                ...        
56702                France
56703              Portugal
56704                France
56705               Germany
56706               Germany
Name: country, Length: 56707, dtype: object

In [277]:
# Aplicar la función para crear una nueva columna 'pais'
bookings['country_name']=bookings['country'].apply(get_country_name)

# Imprimir el DataFrame resultante
print(bookings)


              hotel  is_canceled  lead_time  arrival_date_year  \
0      Resort Hotel            0        109               2016   
1      Resort Hotel            0        109               2016   
2      Resort Hotel            1          2               2016   
3      Resort Hotel            0         88               2016   
4      Resort Hotel            1         20               2016   
...             ...          ...        ...                ...   
56702    City Hotel            0         17               2016   
56703    City Hotel            0        177               2016   
56704    City Hotel            0        107               2016   
56705    City Hotel            0        310               2016   
56706    City Hotel            0        310               2016   

      arrival_date_month  arrival_date_day_of_month  stays_in_weekend_nights  \
0                January                          1                        0   
1                January                       

In [278]:
bookings['country_name'].value_counts()

country_name
Portugal                   22321
United Kingdom              5585
France                      5165
Spain                       4024
Germany                     3865
                           ...  
Burundi                        1
Isle of Man                    1
Syrian Arab Republic           1
Virgin Islands, British        1
Honduras                       1
Name: count, Length: 149, dtype: int64

In [279]:
bookings.to_csv('./bookings_con_pais.csv', index=False, sep=';')


In [281]:
pycountry.countries.search_fuzzy('Timor')

[Country(alpha_2='TL', alpha_3='TLS', flag='🇹🇱', name='Timor-Leste', numeric='626', official_name='Democratic Republic of Timor-Leste')]

In [283]:
import pandas as pd

# Crear un DataFrame de ejemplo
data = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9], 'D': [10, 11, 12]}
df = pd.DataFrame(data)


# Especificar el nuevo orden de las columnas
nuevo_orden = ['A', 'B', 'D', 'C']

# Reordenar el DataFrame con el nuevo orden de las columnas
df = df[nuevo_orden]

# Imprimir el DataFrame resultante
print(df)


   A  B   D  C
0  1  4  10  7
1  2  5  11  8
2  3  6  12  9


In [284]:
country_codes = bookings['country'].unique()

In [285]:
country_codes

array(['RUS', 'PRT', 'ARG', 'FRA', 'GBR', 'DEU', 'BRA', 'IRL', 'USA',
       'KOR', 'ESP', 'DNK', 'CHN', 'NLD', 'HUN', nan, 'POL', 'CN', 'BEL',
       'LUX', 'AUT', 'CHE', 'ARE', 'ITA', 'FIN', 'TUN', 'NOR', 'ROU',
       'JAM', 'ALB', 'AUS', 'SWE', 'HRV', 'HKG', 'IND', 'ISR', 'CZE',
       'IRN', 'GEO', 'DZA', 'MAR', 'AND', 'TUR', 'MOZ', 'ZAF', 'GIB',
       'URY', 'BLR', 'EST', 'LTU', 'JEY', 'CAF', 'CYP', 'GRC', 'LVA',
       'COL', 'GGY', 'KWT', 'CUB', 'UKR', 'CMR', 'MYS', 'THA', 'NZL',
       'BIH', 'MUS', 'NGA', 'COM', 'OMN', 'SUR', 'MEX', 'CHL', 'UGA',
       'BGR', 'CIV', 'SRB', 'DOM', 'JOR', 'SYR', 'PHL', 'PRI', 'JPN',
       'SGP', 'ARM', 'BDI', 'AGO', 'SAU', 'LBN', 'VNM', 'AZE', 'SVN',
       'PLW', 'QAT', 'EGY', 'PER', 'SVK', 'CPV', 'MDV', 'MLT', 'MWI',
       'ECU', 'VEN', 'IRQ', 'SEN', 'IDN', 'TWN', 'HND', 'RWA', 'PAK',
       'ZMB', 'KHM', 'MCO', 'BGD', 'ISL', 'UZB', 'KAZ', 'IMN', 'TJK',
       'NIC', 'BEN', 'MAC', 'VGB', 'CRI', 'TZA', 'GAB', 'MKD', 'GHA',
       'TMP', 'G

In [286]:
bookings

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date,country_name
0,Resort Hotel,0,109,2016,January,1,0,1,2,0,...,RUS,0,0,0,A,D,2.0,Check-Out,02/01/2016,Russian Federation
1,Resort Hotel,0,109,2016,January,1,0,1,2,2,...,RUS,0,0,0,H,H,2.0,Check-Out,02/01/2016,Russian Federation
2,Resort Hotel,1,2,2016,January,1,0,1,2,0,...,PRT,0,0,0,D,D,2.0,No-Show,01/01/2016,Portugal
3,Resort Hotel,0,88,2016,January,1,0,2,2,0,...,ARG,0,0,0,A,D,2.0,Check-Out,03/01/2016,Argentina
4,Resort Hotel,1,20,2016,January,1,0,2,2,2,...,PRT,0,0,0,G,G,2.0,Canceled,22/12/2015,Portugal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,City Hotel,0,17,2016,December,30,2,5,2,0,...,FRA,0,0,0,A,A,9.0,Check-Out,06/01/2017,France
56703,City Hotel,0,177,2016,December,30,2,5,0,0,...,PRT,0,0,0,E,K,NaN,Check-Out,06/01/2017,Portugal
56704,City Hotel,0,107,2016,December,31,2,5,2,0,...,FRA,0,0,0,A,B,2.0,Check-Out,07/01/2017,France
56705,City Hotel,0,310,2016,December,27,2,10,2,0,...,DEU,0,0,0,A,A,9.0,Check-Out,08/01/2017,Germany


In [327]:
def create_arrival_date(df):
    #month dictionary
    months_to_numbers = {
        'Enero': 1, 
        'Febrero': 2, 
        'Marzo': 3, 
        'Abril': 4, 
        'Mayo': 5, 
        'Junio': 6, 
        'Julio': 7, 
        'Agosto': 8, 
        'Septiembre': 9, 
        'Octubre': 10, 
        'Noviembre': 11, 
        'Diciembre': 12
    }

    #mapping month names to numbers
    df['arrival_date_month'] = df['arrival_date_month'].map(months_to_numbers)
    
    #combines the year, month and day columns into a new column 'date'
    df['arrival_date']=pd.to_datetime(df[['arrival_date_year','arrival_date_month','arrival_date_day_of_month']])
    return df

In [328]:
print(type(str(bookings['arrival_date_day_of_month'])))
print(type(bookings['arrival_date_month'][0]))
print(type(str(bookings['arrival_date_year'][0])))

<class 'str'>
<class 'numpy.float64'>
<class 'str'>


In [329]:
create_arrival_date(bookings)

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing

In [ ]:
def get_departure_date():
    